In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

file_path_1024 = glob.glob('/home/asabzi/data/5-exp/10g-NIC/log_1024*')[0]
file_path_2048 = glob.glob('/home/asabzi/data/5-exp/10g-NIC/log_2048*')[0]

df_1024 = pd.read_csv(file_path_1024,names=['softstamp', 'dummy', 'hardstamp'], index_col=None)
df_2048 = pd.read_csv(file_path_2048,names=['softstamp', 'dummy', 'hardstamp'], index_col=None) 
df_1024['hardstamp']=df_1024['hardstamp'].str[:-1].str.replace('.','')
df_2048['hardstamp']=df_2048['hardstamp'].str[:-1].str.replace('.','')
df_rtx_1_1024 = df_1024.iloc[:-1,[2]].reset_index()
df_rtx_2_1024 = df_1024.iloc[1:,[2]].reset_index()

df_rtx_1_2048 = df_2048.iloc[:-1,[2]].reset_index()
df_rtx_2_2048 = df_2048.iloc[1:,[2]].reset_index()

df_diffs_1024 = df_rtx_2_1024['hardstamp'].astype('float64')- df_rtx_1_1024['hardstamp'].astype('float64')
df_diffs_2048 = df_rtx_2_2048['hardstamp'].astype('float64')- df_rtx_1_2048['hardstamp'].astype('float64')

smean_1024 = df_diffs_1024.mean()/1000.0
sstd_1024 = df_diffs_1024.std()/1000.0 
smean_2048 = df_diffs_2048.mean()/1000.0
sstd_2048 = df_diffs_2048.std()/1000.0 

nominal_interval = float(10)
key = '10us'
print("----------------------------------")
print("The average of intervals for %s is: [1024: %.3f, 2048: %.3f] us" % (key, smean_1024, smean_2048))
print("The std of intervals for %s is: [1024: %.3f, 2048: %.3f] us" % (key, sstd_1024, sstd_2048))
print("Time drift percentage is: [1024: %.3f, 2048: %.3f]" % (abs(smean_1024 - nominal_interval)/nominal_interval * 100,abs(smean_2048 - nominal_interval)/nominal_interval * 100) + "%")
print("----------------------------------")

pkt_size = 1024  
rate = (pkt_size * 8 * 1e9)/(df_1024['hardstamp'].astype('float64').max() - df_1024['hardstamp'].astype('float64').min())
print(rate)

----------------------------------
The average of intervals for 10us is: [1024: 249.357, 2048: 304.419] us
The std of intervals for 10us is: [1024: 12.948, 2048: 28.198] us
Time drift percentage is: [1024: 2393.572, 2048: 2944.186]%
----------------------------------
3299.102736842843


## Revisited 4th Experiment

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [30]:
datasets_nolimit = {}
datasets_100limit = {}

keys = []
intervals = (5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 150, 200)
itrs = range(1,6)
for interval in intervals:
    interval_str = str(interval)
    key = interval_str 
    keys.append(key)
    values_per_key_nolimit = []
    values_per_key_100limit = []
    for itr in itrs:
        itr_str = str(itr)
        file_path_nolimit = glob.glob('/home/asabzi/data/revisited-4-exp/10g-NIC/userspace_ts/interval_' + interval_str + 'us_' + itr_str + '_*.csv')[0]
        file_path_100limit = glob.glob('/home/asabzi/data/revisited-5-exp/10g-NIC/userspace_ts/interval_' + interval_str + 'us_' + itr_str + '_*.csv')[0]

        single_value_nolimit = pd.read_csv(file_path_nolimit,names=['softstamp', 'dummy', 'hardstamp', 'before_send', 'after_ack'], index_col=None, dtype='int64' )
        single_value_100limit = pd.read_csv(file_path_100limit,names=['softstamp', 'dummy', 'hardstamp', 'before_send', 'after_ack'], index_col=None, dtype='int64')
        #single_value.sort_values(by=['hardstamp'], inplace=True)
        single_value_100limit.drop(single_value_100limit[single_value_100limit['hardstamp']==0].index, inplace=True)
        single_value_nolimit.drop(single_value_nolimit[single_value_nolimit['hardstamp']==0].index, inplace=True)
        tmp_df = single_value_nolimit['after_ack']-single_value_nolimit['before_send']
        if(key=='150'):
            display(tmp_df.tail())

        values_per_key_nolimit.append(single_value_nolimit)
        values_per_key_100limit.append(single_value_100limit)
    datasets_nolimit[key] = values_per_key_nolimit 
    datasets_100limit[key] = values_per_key_100limit

stas_set_nolimit = {}
for key in keys:
    dfs = datasets_nolimit[key]
    stats = []
    means = []
    sw_delay_means = []
    stds = []
    drifts = []
    for df in dfs: 
        nominal_interval = float(key)  
        df_rtx_1 = df.iloc[:-1,[2]].reset_index()
        df_rtx_2 = df.iloc[1:,[2]].reset_index()
        df_diffs = (df_rtx_2['hardstamp']- df_rtx_1['hardstamp'])* 1e-3
        sw_delay_mean = (df['after_ack']- df['before_send']).mean() * 1e-3
        sw_delay_means.append(sw_delay_mean)
        smean = df_diffs.mean()
        sstd = df_diffs.std() 
        drift = abs(smean - nominal_interval) 
        means.append(smean)
        stds.append(sstd)
        drifts.append(drift)
        #if(key == '1000'):
        #    display(df_diffs.max())
        print("----------------------------------")
        print("The average of intervals for %s us is:  %.3f us" % (key, smean))
        print("The std of intervals for %s us is: %.3f us" % (key, sstd))
        print("Time drift absolute value is: %.3f" % (drift))
        print("Time delay absolute value is: %.3f" % (sw_delay_mean))
    stats.append(sum(means)/len(means))
    stats.append(sum(stds)/len(stds))
    stats.append(sum(drifts)/len(drifts))
    stats.append(sum(sw_delay_means)/len(sw_delay_means))
    stas_set_nolimit[key] = stats



stas_set_100limit = {}
for key in keys:
    dfs = datasets_100limit[key]
    stats = []
    means = []
    sw_delay_means = []
    stds = []
    drifts = []
    for df in dfs: 
        nominal_interval = float(key)  
        df_rtx_1 = df.iloc[:-1,[2]].reset_index()
        df_rtx_2 = df.iloc[1:,[2]].reset_index()
        df_diffs = (df_rtx_2['hardstamp']- df_rtx_1['hardstamp'])* 1e-3
        sw_delay_mean = (df['after_ack']- df['before_send']).mean() * 1e-3
        sw_delay_means.append(sw_delay_mean)
        smean = df_diffs.mean()
        sstd = df_diffs.std() 
        drift = abs(smean - nominal_interval) 
        means.append(smean)
        
        stds.append(sstd)
        drifts.append(drift)
        #if(key == '1000'):
        #    display(df_diffs.max())
        print("----------------------------------")
        print("The average of intervals for %s us is:  %.3f us" % (key, smean))
        print("The std of intervals for %s us is: %.3f us" % (key, sstd))
        print("Time drift absolute value is: %.3f" % (drift))
        print("Time delay absolute value is: %.3f" % (sw_delay_mean))
    stats.append(sum(means)/len(means))
    stats.append(sum(stds)/len(stds))
    stats.append(sum(drifts)/len(drifts))
    stats.append(sum(sw_delay_means)/len(sw_delay_means))
    stas_set_100limit[key] = stats



9995    25
9996    24
9997    32
9998    31
9999    32
dtype: int64

9995    25
9996    24
9997    23
9998    25
9999    23
dtype: int64

9995    27
9996    23
9997    24
9998    32
9999    32
dtype: int64

9995    26
9996    24
9997    24
9998    33
9999    25
dtype: int64

9995    24
9996    23
9997    24
9998    23
9999    23
dtype: int64

----------------------------------
The average of intervals for 5 us is:  86.918 us
The std of intervals for 5 us is: 3.988 us
Time drift absolute value is: 81.918
Time delay absolute value is: 99.924
----------------------------------
The average of intervals for 5 us is:  86.963 us
The std of intervals for 5 us is: 3.396 us
Time drift absolute value is: 81.963
Time delay absolute value is: 99.924
----------------------------------
The average of intervals for 5 us is:  86.936 us
The std of intervals for 5 us is: 3.655 us
Time drift absolute value is: 81.936
Time delay absolute value is: 0.024
----------------------------------
The average of intervals for 5 us is:  86.942 us
The std of intervals for 5 us is: 3.288 us
Time drift absolute value is: 81.942
Time delay absolute value is: 0.024
----------------------------------
The average of intervals for 5 us is:  86.933 us
The std of intervals for 5 us is: 3.045 us
Time drift absolute value is: 81.933
Time delay absolute value is: 99.9

In [24]:
#print(stas_set)
for key in keys:
    print("------ %sus -------" % (key))
    print("Mean: [NoLimit: %.4f, WithLimit: %.4f]" % (stas_set_nolimit[key][0], stas_set_100limit[key][0]))
    print("std: [NoLimit: %.4f, WithLimit: %.4f]" % (stas_set_nolimit[key][1], stas_set_100limit[key][1]))
    print("Drift: [NoLimit: %.4f, WithLimit: %.4f]" % (stas_set_nolimit[key][2], stas_set_100limit[key][2]))
    print("SW delay: [NoLimit: %.4f, WithLimit: %.4f]" % (stas_set_nolimit[key][3], stas_set_100limit[key][3]))

    #print("Std: %.4f" % (stas_set[key][1]))   
    #print("Drift: %.4f" % (stas_set[key][2]))   
   

------ 5us -------
Mean: [NoLimit: 86.9383, WithLimit: 2223.9489]
std: [NoLimit: 3.4744, WithLimit: 4764.2072]
Drift: [NoLimit: 81.9383, WithLimit: 2218.9489]
SW delay: [NoLimit: 59.9638, WithLimit: 0.1006]
------ 10us -------
Mean: [NoLimit: 92.3617, WithLimit: 1528.1050]
std: [NoLimit: 3.3790, WithLimit: 3602.6677]
Drift: [NoLimit: 82.3617, WithLimit: 1518.1050]
SW delay: [NoLimit: 39.9841, WithLimit: 0.0806]
------ 20us -------
Mean: [NoLimit: 102.4385, WithLimit: 1039.5751]
std: [NoLimit: 2.9886, WithLimit: 2065.7078]
Drift: [NoLimit: 82.4385, WithLimit: 1019.5751]
SW delay: [NoLimit: 20.0041, WithLimit: 0.0419]
------ 30us -------
Mean: [NoLimit: 112.2310, WithLimit: 210.2560]
std: [NoLimit: 3.8197, WithLimit: 22.5404]
Drift: [NoLimit: 82.2310, WithLimit: 180.2560]
SW delay: [NoLimit: 20.0041, WithLimit: 0.0176]
------ 40us -------
Mean: [NoLimit: 122.4786, WithLimit: 223.6845]
std: [NoLimit: 4.1278, WithLimit: 24.2745]
Drift: [NoLimit: 82.4786, WithLimit: 183.6845]
SW delay: [NoL